# Extended Kalman filter with Parameter Identification for Nomoto model
An Extended Kalman filter with a Nomoto model as the predictor will be developed. 
The filter should also estimate the parameters in the Nomoto model. This is done by setting up a system where the parameters are defined as some of the states in the model 
The filter is run on simulated data as well as real model test data.

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import sympy as sp

import vessel_manoeuvring_models.visualization.book_format as book_format
book_format.set_style()
from vessel_manoeuvring_models.substitute_dynamic_symbols import lambdify
from sympy import Matrix
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify
from sympy.physics.vector.printing import vpprint, vlatex
from vessel_manoeuvring_models.data import mdl
from vessel_manoeuvring_models.kalman_filter import extended_kalman_filter

## Nomoto model for ship manoeuvring dynamics
The Nomoto model can be written as:

In [ ]:
r,r1d,r2d = sp.symbols('r \dot{r} \ddot{r}')
psi,psi1d = sp.symbols('psi \dot{\psi}')
h,u = sp.symbols('h u')
x, x1d = sp.symbols('x \dot{x}')
A,B,C,D,E, Phi = sp.symbols('A B C D E Phi')
w = sp.symbols('w')

K, delta, T_1, T_2 = sp.symbols('K delta T_1 T_2')

eq_nomoto = sp.Eq(K*delta,
                 r + T_1*r1d + T_2*r2d)
Math(vlatex(eq_nomoto))

where $r$ is yaw rate with its time derivatives and $\delta$ is the rudder angle. $K$, $T_{1}$
 and $T_{1}$ are the coefficients describing the hydrodynamics of the ship.
 
For slow manoeuvres this equation can be further simplified by removing the $\ddot{r}$ term into a first order Nomoto model:

In [ ]:
eq_nomoto_simple = eq_nomoto.subs(r2d,0)
Math(vlatex(eq_nomoto_simple))

### Simulation model

In [ ]:
f_hat = sp.Function('\hat{f}')(x,u,w)
eq_system = sp.Eq(x1d, f_hat)
eq_system

Where the state vector $x$:

In [ ]:
eq_x = sp.Eq(x, sp.UnevaluatedExpr(Matrix([psi,r])))
eq_x

and input vector $u$:

and $w$ is zero mean Gausian process noise

For the nomoto model the time derivatives for the states can be expressed as:

In [ ]:
eq_psi1d = sp.Eq(psi1d,r)
eq_psi1d

In [ ]:
eq_r1d = sp.Eq(r1d,sp.solve(eq_nomoto_simple,r1d)[0])
eq_r1d

In [ ]:
def lambda_f_constructor(K, T_1):
    def lambda_f(x, u):
        delta = u
        f = np.array([[x[1], (K*delta-x[1])/T_1]]).T
        return f

    return lambda_f

## Simulation
Simulation with this model where rudder angle shifting between port and starboard

In [ ]:
T_1_ = 1.8962353076056344
K_ = 0.17950970687951323
h_ = 0.01

lambda_f = lambda_f_constructor(K=K_, T_1=T_1_)

In [ ]:
def simulate(E, ws, t, us):
    
    simdata = []
    x_=np.deg2rad(np.array([[0,0]]).T)
    
    for u_,w_ in zip(us,ws):
               
        x_=x_ + h_*lambda_f(x=x_.flatten(), u=u_)
    
        simdata.append(x_.flatten())
        
    simdata = np.array(simdata)
    df = pd.DataFrame(simdata, columns=["psi","r"], index=t)
    df['delta'] = us
    
    return df

In [ ]:
N_ = 8000

t_ = np.arange(0,N_*h_,h_)
    
us = np.deg2rad(np.concatenate((-10*np.ones(int(N_/4)),
                                10*np.ones(int(N_/4)),
                                -10*np.ones(int(N_/4)),
                                10*np.ones(int(N_/4)))))

np.random.seed(42)
E = np.array([[0, 1]]).T
process_noise = np.deg2rad(0.01)
ws = process_noise*np.random.normal(size=N_)
df = simulate(E=E, ws=ws, t=t_, us=us)

measurement_noise = np.deg2rad(0.5)
df['epsilon'] = measurement_noise*np.random.normal(size=N_)
df['psi_measure'] = df['psi'] + df['epsilon']
df['psi_deg'] = np.rad2deg(df['psi'])
df['psi_measure_deg'] = np.rad2deg(df['psi_measure'])
df['delta_deg'] = np.rad2deg(df['delta'])


In [ ]:
fig,ax=plt.subplots()
df.plot(y='psi_deg', ax=ax)
df.plot(y='psi_measure_deg', ax=ax, zorder=-1)
df.plot(y='delta_deg', ax=ax, zorder=-1)
df.plot(y='r')
ax.set_title('Simulation with measurement and process noise')
ax.set_xlabel('Time [s]');



## Kalman filter
Implementation of the Kalman filter. The code is inspired of this Matlab implementation: [ExEKF.m](https://github.com/cybergalactic/MSS/blob/master/mssExamples/ExEKF.m).

In [ ]:
jac = sp.eye(3,3) + Matrix([r,eq_r1d.rhs,0]).jacobian([psi,r,T_1])*h
jac

In [ ]:
def lambda_f_constructor2(K):
    def lambda_f(x, u):
        delta = u
        T_1 = x[2]  # Note! T_1 is the third state now!
        r = x[1]
        
        f = np.array([[r, (K*delta-r)/T_1, 0]]).T
        return f

    return lambda_f

In [ ]:
def lambda_jacobian_constructor(h, K):
    def lambda_jacobian(x, u):

        T_1 = x[2]  # Note! T_1 is the third state now!
        delta = u
        r = x[1]

        jac = np.array(
            [
                [1, h, 0],
                [0, 1 - h / T_1, -h * (K * delta - r) / T_1 ** 2],
                [0, 0, 1],
            ]
        )
        return jac

    return lambda_jacobian

In [ ]:
lambda_jacobian = lambda_jacobian_constructor(h=h_, K=K_)
lambda_f = lambda_f_constructor2(K=K_)

In [ ]:
lambda_jacobian(x=[0,0,0.1], u=0)

In [ ]:
lambda_f(x=[0,0,0.1], u=0)

In [ ]:
x0=np.deg2rad(np.array([[0,0,3]]).T)
P_prd = np.diag([np.deg2rad(1), np.deg2rad(0.1), 0.1])

Qd = np.diag([np.deg2rad(0), 2])

Rd = np.deg2rad(1)

ys = df['psi_measure'].values

E_ = np.array(
    [
     [0,0], 
     [1,0],
     [0,1]
    ],
)

C_ = np.array([[1, 0, 0]])

Cd_ = C_
Ed_ = h_ * E_

time_steps = extended_kalman_filter(x0=x0, P_prd=P_prd, lambda_f=lambda_f, 
                                    lambda_jacobian=lambda_jacobian,h=h_, us=us, ys=ys, E=E_, Qd=Qd, Rd=Rd, Cd=Cd_)
x_hats = np.array([time_step["x_hat"] for time_step in time_steps]).T
time = np.array([time_step["time"] for time_step in time_steps]).T
Ks = np.array([time_step["K"] for time_step in time_steps]).T
stds = np.sqrt(np.array([[time_step["P_hat"][0,0],
                          time_step["P_hat"][1,1],
                          time_step["P_hat"][2,2]] for time_step in time_steps]).T)




In [ ]:
n=len(P_prd)
fig,axes=plt.subplots(nrows=n)

df['T_1'] = T_1_
keys = ['psi','r','T_1']
for i,key in enumerate(keys):
    
    ax=axes[i]
    df.plot(y=key, ax=ax, label="True")
    if key=='psi':
        df.plot(y='psi_measure', ax=ax, label="Measured", zorder=-1)
    
    ax.plot(time, x_hats[i, :], "-", label="kalman")
    
    std_top = x_hats[i, :] + stds[i, :]
    std_btm = x_hats[i, :] - stds[i, :]
    ax.plot(time, std_top, linestyle=':', color='k', lw=1, alpha=0.4)
    ax.plot(time, std_btm, linestyle=':', color='k', lw=1, alpha=0.4)
    ax.fill_between(time, std_top, std_btm,
                     facecolor='yellow', alpha=0.2, interpolate=True, label='+/- std')
    
    
    ax.set_ylabel(key)
    ax.legend()
    

In [ ]:
fig,ax=plt.subplots()
for i,key in enumerate(keys):
    ax.plot(time,Ks[i,:],label=key)
ax.set_title('Kalman gains')
ax.legend();
ax.set_ylim(-1,1);


In [ ]:
fig,axes=plt.subplots(nrows=2)

ax=axes[0]
for i,key in enumerate(keys):
    ax.plot(time,stds[i,:]**2,label=key)
ax.set_title('Variances')
ax.legend();

ax=axes[1]
df.plot(y='delta',ax=ax)



# Real data
Using the developed Kalman filter on some real model test data

## Load test

In [ ]:
id=22773
df, units, meta_data = mdl.load(dir_path = '../data/raw', id=id)
df.index = df.index.total_seconds()
df.index-=df.index[0]

In [ ]:
from vessel_manoeuvring_models.visualization.plot import track_plot
fig,ax=plt.subplots()
fig.set_size_inches(10,10)
track_plot(df=df, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);

In [ ]:
ys = df['psi'].values
h_m=h_ = df.index[1]-df.index[0]
us = df['delta'].values

x0=np.deg2rad(np.array([[0,0,T_1_]]).T)
P_prd = np.diag([np.deg2rad(1), np.deg2rad(0.1), 100])

Qd = np.diag([np.deg2rad(5), 10])

Rd = np.deg2rad(0.1)

E_ = np.array(
    [
     [0,0], 
     [1,0],
     [0,1]
    ],
)

C_ = np.array([[1, 0, 0]])

Cd_ = C_
Ed_ = h_ * E_

time_steps = extended_kalman_filter(x0=x0, P_prd=P_prd, lambda_f=lambda_f, 
                                    lambda_jacobian=lambda_jacobian,h=h_, us=us, ys=ys, E=E_, Qd=Qd, Rd=Rd, Cd=Cd_)
x_hats = np.array([time_step["x_hat"] for time_step in time_steps]).T
time = np.array([time_step["time"] for time_step in time_steps]).T
Ks = np.array([time_step["K"] for time_step in time_steps]).T
stds = np.sqrt(np.array([[time_step["P_hat"][0,0],
                          time_step["P_hat"][1,1],
                          time_step["P_hat"][2,2]] for time_step in time_steps]).T)

In [ ]:
n=len(P_prd)
fig,axes=plt.subplots(nrows=n)

ax = axes[0]
df.plot(y='psi', label='Measured', ax=ax)

df['T_1'] = T_1_
keys = ['psi','r','T_1']
for i,key in enumerate(keys):
    
    ax=axes[i]
   
    ax.plot(time, x_hats[i, :], "-", label="kalman")
    
    std_top = x_hats[i, :] + stds[i, :]
    std_btm = x_hats[i, :] - stds[i, :]
    ax.plot(time, std_top, linestyle=':', color='k', lw=1, alpha=0.4)
    ax.plot(time, std_btm, linestyle=':', color='k', lw=1, alpha=0.4)
    ax.fill_between(time, std_top, std_btm,
                     facecolor='yellow', alpha=0.2, interpolate=True, label='+/- std')
    
    
    ax.set_ylabel(key)
    ax.legend()

In [ ]:
fig,ax=plt.subplots()
for i,key in enumerate(keys):
    ax.plot(time,Ks[i,:],label=key)
ax.set_title('Kalman gains')
ax.legend();
ax.set_ylim(-1,1);